### **Llamar el xlsx desde el datastore para convertirlo a csv**

In [17]:
from azureml.core import Workspace, Dataset, Datastore

ws = Workspace(subscription_id='35d4664b-ed82-4074-b51a-a96056a154ac',
               resource_group='grupopruebatecnica',
               workspace_name='WS-PruebaTecnica')

datastore = Datastore.get(ws, "workspaceblobstore")

dataset = Dataset.File.from_files(path=(datastore, "workspaceblobstore/sentencias_pasadas.xlsx"))

local_path = dataset.download('.', overwrite=True)
local_path

{'infer_column_types': 'False', 'activity': 'download'}
{'infer_column_types': 'False', 'activity': 'download', 'activityApp': 'FileDataset'}


['/mnt/batch/tasks/shared/LS_root/mounts/clusters/johanbolivarmosquera1/code/Users/johanbolivarmosquera/sentencias_pasadas.xlsx']

### **Xlsx transformado ya a Csv**

In [18]:
import zipfile
import xml.etree.ElementTree as ET
import csv

xlsx_path = "sentencias_pasadas.xlsx"
csv_path = "sentencias_pasadas.csv"

# --- Abrir XLSX como ZIP ---
with zipfile.ZipFile(xlsx_path) as z:
    sheet_xml = z.read('xl/worksheets/sheet1.xml')
    shared_xml = z.read('xl/sharedStrings.xml')

# --- Parsear sharedStrings.xml ---
shared_root = ET.fromstring(shared_xml)
ns = {'main': 'http://schemas.openxmlformats.org/spreadsheetml/2006/main'}

shared_strings = []
for si in shared_root.findall('main:si', ns):
    text_elements = si.findall('.//main:t', ns)
    text = ''.join([t.text for t in text_elements if t.text])  # unir si hay varios fragmentos
    shared_strings.append(text)

# --- Parsear sheet1.xml ---
sheet_root = ET.fromstring(sheet_xml)

rows = []
for row in sheet_root.findall('.//main:row', ns):
    values = []
    for cell in row.findall('main:c', ns):
        cell_type = cell.get('t')  # si t="s", es shared string
        value = cell.find('main:v', ns)

        if value is None:
            values.append("")
        else:
            if cell_type == "s":
                # índice a sharedStrings.xml
                idx = int(value.text)
                values.append(shared_strings[idx])
            else:
                # valor directo
                values.append(value.text)
    rows.append(values)

# --- Guardar CSV ---
with open(csv_path, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerows(rows)

print("CSV generado correctamente:", csv_path)

CSV generado correctamente: sentencias_pasadas.csv


In [19]:
import pandas as pd

df = pd.read_csv("sentencias_pasadas.csv", header=None)
df.head()

,0,1,2,3,4,5,6,7
0,#,Relevancia,Providencia,Tipo,Fecha Sentencia,Tema - subtema,resuelve,sintesis
1,1,966965,T-185/22,NaN,44712,NaN,en nombre del pueblo y por mandato de la Const...,En este caso se formula la acción de tutela en...
2,3,963168,T-356/21,NaN,44484,ACCIÓN DE TUTELA PARA PROTEGER EL DERECHO A LA...,en nombre del pueblo y por mandato de la Const...,El peticionario considera que los accionantes ...
3,5,956201,T-351/22,NaN,44841,ACCIÓN DE TUTELA PARA PROTEGER EL DERECHO A LA...,"administrando justicia en nombre del Pueblo, y...",El periodista accionante acusa al abogado acci...
4,6,955889,T-246/21,NaN,44406,ACCION DE TUTELA PARA PROTEGER EL DERECHO A LA...,en nombre del pueblo y por mandato de la Const...,Se presenta la acción de tutela en contra de u...


### **Arreglar formato de fecha**

In [21]:
import pandas as pd
import numpy as np

# -------------------------------
# CARGA DEL ARCHIVO
# -------------------------------
# Si tu archivo NO tiene encabezados reales, usa header=None.
df = pd.read_csv("sentencias_pasadas.csv", header=None)

# Asignar nombres de columnas
df.columns = ["#", "Relevancia", "Providencia", "Tipo", "Fecha", "Tema", "resuelve", "sintesis"]

# -------------------------------
# LIMPIEZA DE LA COLUMNA FECHA
# -------------------------------

# 1. Eliminar filas donde "Fecha" tenga texto tipo "Fecha Sentencia"
df = df[df["Fecha"].astype(str).str.isnumeric()]  # descarta filas extrañas

# 2. Convertir la columna Fecha a número
df["Fecha_num"] = pd.to_numeric(df["Fecha"], errors="coerce")

# 3. Convertir el número Excel a fecha real
df["FechaSentencia"] = pd.to_datetime(df["Fecha_num"], unit="d", origin="1899-12-30")

# 4. Formato dd/mm/yyyy
df["FechaSentencia"] = df["FechaSentencia"].dt.strftime("%d/%m/%Y")

# -------------------------------
# MOSTRAR RESULTADO
# -------------------------------
print(df[["Fecha", "FechaSentencia"]].head(10))

    Fecha FechaSentencia
1   44712     31/05/2022
2   44484     15/10/2021
3   44841     07/10/2022
4   44406     29/07/2021
5   44743     01/07/2022
6   45435     23/05/2024
7   45554     19/09/2024
8   43559     04/04/2019
9   44832     28/09/2022
10  44431     23/08/2021


In [24]:
# Reemplazar columna Fecha con los valores corregidos FechaSentencia
df["Fecha"] = df["FechaSentencia"]

# Eliminar columnas que ya no sirven
df = df.drop(columns=["Fecha_num", "FechaSentencia"])

In [ ]:
df.head()

### **Seleccionar 3 demandas**

In [26]:
# Elegimos 3 demandas (las primeras 3)
tres_casos = df.head(3)

tres_casos[["Providencia", "sintesis"]]

,Providencia,sintesis
1,T-185/22,En este caso se formula la acción de tutela en...
2,T-356/21,El peticionario considera que los accionantes ...
3,T-351/22,El periodista accionante acusa al abogado acci...


In [27]:
for i, row in tres_casos.iterrows():
    print(f"- En el caso {row['Providencia']}, la sentencia fue: {row['sintesis']}\n")

- En el caso T-185/22, la sentencia fue: En este caso se formula la acción de tutela en contra de un particular, alegando la vulneración de los derechos fundamentales al buen nombre y honra, en virtud de las afirmaciones publicadas en redes sociales sobre el procedimiento médico veterinario que se le practicó a la mascota de la accionada, las cuales, según los accionantes, ponen en duda su ética profesional, al ser información errónea e inexacta. La Corte decidió declararla IMPROCEDENTE la acción de tutela, por no acreditar los requisitos de procedencia frente a la libertad de expresión en las redes sociales. En particular, no se demostró el estado de indefensión, como tampoco se superó el requisito de subsidiariedad.

- En el caso T-356/21, la sentencia fue: El peticionario considera que los accionantes violaron sus derechos fundamentales al buen nombre, la honra y la integridad moral, al difundir a través de redes sociales denuncias públicas realizadas en su contra y en las que se le

In [ ]:
tres_casos[["Providencia", "Tema"]]

In [ ]:
for i, row in tres_casos.iterrows():
    print(f"- El caso {row['Providencia']} trató sobre: {row['Tema']}\n")

In [ ]:
acoso = df[df["Tema"].str.contains("acoso escolar", case=False, na=False)]

acoso[["Providencia", "sintesis"]]

In [ ]:
for i, row in acoso.iterrows():
    print(f"- En el caso relacionado con acoso escolar ({row['Providencia']}), el juez decidió lo siguiente: {row['sintesis']}\n")

In [ ]:
acoso[["Tema", "sintesis"]]

### **Verifica si existen casos que hablan sobre el PIAR y devuelve el tema y sus sentencias**

In [40]:
piar = df[
    df.apply(lambda row:
             row.astype(str).str.contains("PIAR", case=False, na=False).any(),
             axis=1)
]

In [ ]:
# Si no encuentra devuelve 
if len(piar) == 0:
    print("No se encontraron casos relacionados con PIAR.")
else:
    # En caso que encuentre devuelve
    for i, row in piar.iterrows():
        print(f"- Hubo un caso relacionado con PIAR ({row['Providencia']}).")
        print(f"  Se trató sobre: {row['Tema']}")
        print(f"  La sentencia fue: {row['sintesis']}\n")

### **Busca una fila especifica del df y la devuelve completa**

In [ ]:
with pd.option_context('display.max_colwidth', None):
    print(df.loc[100])